In [24]:
# quitely pip install  necessary libraries
!pip install -q osmnx
!pip install -q mapclassify
!pip install -q beautifulsoup4

In [25]:
"""takes address as input to prompt and
converts it to coordinates and a geodataframe
author: rayne davidson"""


from pandas.core.dtypes.cast import invalidate_string_dtypes
import pandas as pd
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point
import matplotlib as mpl
import folium
import matplotlib.pyplot as plt
from mapclassify import classify
import numpy as np
import string

# mapping lowercase cardinal directions (in the event user enters as lowercase)
# to a dictionary of lowercase to uppercase cardinal directions. this only needs
# to be done once as a global list so I dont think it needs to be in a function
directions_list = ["n", "ne", "nw", "s", "se", "sw", "e", "w"]
upper_directions = [x.upper() for x in directions_list]
DIRECTIONS_DICT = dict(zip(directions_list, upper_directions))


def user_input(prompt):
  """asks user for given prompt"""
  return input(prompt)


def user_address_input():
  """asks user for address. does not move address forward until address is
  confirmed by user"""
  prompt = "Address for evaluation: "
  address = format_address(user_input(prompt))
  while confirmation(address) == False:
    address = re_entry()
  return address


def user_confirm(address):
  """asks user to confirm given address (in event of typo). address is formatted
  when given back to user to look nicer"""
  confirm_prompt = f'Please confirm that the address {address} is correct (Y/N): '
  confirmation = user_input(confirm_prompt)
  return confirmation


def format_address(address):
  """formats address to capitalize cardinal directions and first letter
  of road names - chat gpt helped with this"""
  address_individuals = address.split()
  formatted_words = []
  for word in address_individuals:
    if word.lower() in DIRECTIONS_DICT:
      formatted_words.append(DIRECTIONS_DICT[word.lower()])
    else:
      formatted_words.append(word.capitalize())
  formatted_address = ' '.join(formatted_words)
  return formatted_address


def confirmation(address):
  """checks if user confirms address or if address should be re-entered.
  returns true or false"""
  valid_confirmations = ["yes", "y", "Yes", "Y"]
  confirmation = user_confirm(address)
  if confirmation in valid_confirmations:
    return True
  elif confirmation not in valid_confirmations:
    return False


def re_entry():
  """prompts user to re-enter address if confirmation is denied"""
  reenter_prompt = "Please re-enter address: "
  address = format_address(user_input(reenter_prompt))
  return address


def geocode_address(address):
  """geocodes address so it can be explored in gpd"""
  try:
    geocode_result = ox.geocode(address)
    return geocode_result
  except:
    return None


def valid_geocode(geocode_result):
  """returns Point geometry for a valid geocoded address"""
  y, x = geocode_result
  point = Point(x, y)  # Create a Point geometry
  return point


def address_to_gdf(address, point):
  """creates geodataframe from geocoded address"""
  pr_crs = 4326
  gdf = gpd.GeoDataFrame({'Address': [address]}, geometry=[point], crs=pr_crs)
  return gdf


def invalid_geocode(geocode_result):
  """if address is not able to be geocoded, prompts to try again
  until address is successfully geocoded"""
  while geocode_result is None:
      print("No valid address found. Please try again")
      print("Try entering just street address (no city/state)")
      prompt = "Please re-enter address: "
      address = format_address(user_input(prompt))
      while confirmation(address) == False:
        address = re_entry()
      geocode_result = geocode_address(address)
  point = valid_geocode(geocode_result)
  print("New address is successfully validated!")
  return point


def geocode(address):
  """applies geocoding to address. Moves to invalid geocode re-entry prompt
  if address is not able to be geocoded"""
  geocode_result = geocode_address(address)
  if geocode_result is not None:
    point = valid_geocode(geocode_result)
    print("Address is successfully validated!")
    return point
  elif geocode_result is None:
    point = invalid_geocode(geocode_result)
    return point


def confirmed_address():
  """moving on with valid address and global coordinates for versatility"""
  global ADDRESS
  ADDRESS = user_address_input()
  point = geocode(ADDRESS)
  address_gdf = address_to_gdf(ADDRESS, point)
  return address_gdf

# Download data (Web scraping)

In [26]:
"""web scraping for tsunami, liquifaction, volcano and geology data
from the WA state Dept of Natural Resources (DNR)"""

import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def set_soup(url_link):
  url = url_link
  response = requests.get(url)

  if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      return soup
  else:
      print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
      exit()

def soup_find(hazard):
  """finds soup links of a peticular hazard"""
  links = soup.find_all('a', string=hazard)
  return links

def get_url_stuff(link):
  """downloads stuff from the link(s) to pwd"""
  absolute_url = urljoin(url, link['href'])
  filename = os.path.basename(absolute_url)
  file_response = requests.get(absolute_url)
  if file_response.status_code == 200:
      with open(filename, 'wb') as file:
          file.write(file_response.content)
      print(f"Downloaded {filename}")
  else:
      print(f"Failed to download {filename}. Status code: {file_response.status_code}")

def download_data(url_link, hazard):
  """downloads relevant data from the relevant place"""
  soup = set_soup(url_link)
  links = soup_find(hazard)
  for link in links:
    get_url_stuff(link)

def download_haz_data():
  """downloads hazard data from specific place (WA DNR)"""
  url_link = 'https://www.dnr.wa.gov/programs-and-services/geology/publications-and-data/gis-data-and-databases'

  # all of these are the link names on the DNR website for the relevant data I need
  hazards_lst = ["Tsunami Hazard", "Ground Response", "Simplified Volcanic Hazards", "1:500,000 scale"]
  for hazard in hazards_lst:
    download_data(url_link, hazard)

# download_haz_data() # i intetionally have this turned off because i dont need to download it again

Running the web scraping wont actually be very useful for you, as I had to run this and open the files in ArcGIS Pro to save them as shapefiles that can be used in geopandas. The only pythoning way around this was using arcpy which requires an ArcGIS Pro subscription to be logged in/function, which i wasn't sure how well that would work when submitted/used elsewhere, so I opted for doing it this way


**you will need to upload the shapefile data before you can proceed**

Source: https://www.dnr.wa.gov/programs-and-services/geology/publications-and-data/gis-data-and-databases

# Read in the Hazards Data

I attempted to make a GitHub repository that the data could be pulled from but had a lot of issues making it work (including file sizes too large)

In [27]:
tsunami = gpd.read_file("TsunamiHazardAreas.shp")
liquif = gpd.read_file("liquifaction_susceptibility.shp")
volcano = gpd.read_file("volcano.shp")
geology = gpd.read_file("geology.shp")

# Data description dictionaries

In [28]:
"""i could have left at least most of the descriptions that came with
the data, but the reason i am making this project at all is because the data
from the goverment can be difficult to obtain and/or understand, so this
was my attempt at making the hazards more understandable for normal people"""

# TSUNAMI
# ------------------------------------
# descriptions derrived from hazard.TSUNAMI_SC.unique() list, but put in more user-friendly readability (simplified)
tsunami_dict = (
    {
      tsunami.TSUNAMI_SC.unique()[0]
      :"Possible tsunami inundation in 9.0 Cascadia Fault Earthquake",
      tsunami.TSUNAMI_SC.unique()[1]
      :"Possible tsunami inundation in 7.3 Tacoma Fault Earthquake",
      tsunami.TSUNAMI_SC.unique()[2]
      :"Possible tsunami inundation in 7.3 Tacoma Fault Earthquake",
      tsunami.TSUNAMI_SC.unique()[3]
      :"Possible tsunami inundation in 9.0 Cascadia Fault Earthquake",
      tsunami.TSUNAMI_SC.unique()[4]
      :"Possible tsunami inundation in 7.5 Cascadia Fault Earthquake",
      tsunami.TSUNAMI_SC.unique()[5]
      :"Possible tsunami inundation in 9.1 Cascadia Fault Earthquake",
      tsunami.TSUNAMI_SC.unique()[6]
      :"Possible tsunami inundation in 9.1 Cascadia Fault Earthquake",
      tsunami.TSUNAMI_SC.unique()[7]
      :"Possible tsunami inundation - unspecified magitude or fault",
      tsunami.TSUNAMI_SC.unique()[8]
      :"Possible tsunami inundation in 9.1 Cascadia Fault Earthquake",
      tsunami.TSUNAMI_SC.unique()[9]
      :"Possible coastal, estuarine and waterway inundation; not analyzed"
    }
)

# LIQUIFACTION
# ------------------------------------
# descriptions derrived from liquif.LIQUEFAC_1.unique() list, but put in more user-friendly readability (simplified)
liquif_dict = (
    {
      liquif.LIQUEFAC_1.unique()[0]:"No risk of liquefaction",
      liquif.LIQUEFAC_1.unique()[1]:"High risk of liquefaction",
      liquif.LIQUEFAC_1.unique()[2]:"No risk of liquefaction",
      liquif.LIQUEFAC_1.unique()[3]:"Low risk of liquefaction",
      liquif.LIQUEFAC_1.unique()[4]:"Low to Moderate risk of liquefaction",
      liquif.LIQUEFAC_1.unique()[5]:"Moderate risk of liquefaction",
      liquif.LIQUEFAC_1.unique()[6]:"Moderate to High risk of liquefaction",
      liquif.LIQUEFAC_1.unique()[7]:"No risk of liquefaction",
      liquif.LIQUEFAC_1.unique()[8]:"Very Low risk of liquefaction",
      liquif.LIQUEFAC_1.unique()[9]:"Very Low to Low risk of liquefaction",
      liquif.LIQUEFAC_1.unique()[10]:"No risk of liquefaction"
    }
)

# VOLCANO
# ------------------------------------
# descriptions derrived from volcano.HAZARD_TYP.unique() list, but put in more user-friendly readability (simplified)
volcano_risk_dict = (
    {
      volcano.HAZARD_TYP.unique()[0]:"Lahar risk zone",
      volcano.HAZARD_TYP.unique()[1]:"Near-volcano hazards zone",
      volcano.HAZARD_TYP.unique()[2]:"Regional lava flow risk zone",
      volcano.HAZARD_TYP.unique()[3]:"Tephra ash risk zone",
      volcano.HAZARD_TYP.unique()[4]:"Lahar sediment risk zone",
    }
)

# the volcano data has a list of actions to do in the event of each hazard
# i feel as though that information is important (not just knowing the hazard)
v_haz = volcano.drop_duplicates(subset=["HAZARD_TYP"])
v_haz = v_haz.drop(["VOLCANO", "DESCRIPTIO", "HYPERLINK", "DEFINITION",
                    "Shape_Leng", "Shape_Area", "geometry"], axis=1)

haz_actions = v_haz.values.tolist()
actions = [i[1:] for i in haz_actions]

# descriptions derrived from volcano.HAZARD_TYP.unique() list, but put in more user-friendly readability (simplified)
volcano_actions_dict = (
    {
      volcano.HAZARD_TYP.unique()[0]:[actions[0][0], actions[0][1],
                                      actions[0][2], actions[0][3]],
      volcano.HAZARD_TYP.unique()[1]:[actions[1][0], actions[1][1],
                                      actions[1][2], actions[1][3]],
      volcano.HAZARD_TYP.unique()[2]:[actions[2][0], actions[2][1],
                                      actions[2][2], actions[2][3]],
      volcano.HAZARD_TYP.unique()[3]:[actions[3][0], actions[3][1],
                                      actions[3][2], actions[3][3]],
      volcano.HAZARD_TYP.unique()[4]:[actions[4][0], actions[4][1],
                                      actions[4][2], actions[4][3]],
    }
)


# GEOLOGY (not a hazard just fun to know because i'm a geologist)
# ------------------------------------
# I did do this manually from the unit names given in the ArcGIS file because they only used labels inside arc to name,
# rather than having an the name of the geologic units as actual data. yes it was a pain in the foot

labels = []
for i in geology.Label.unique():
  labels.append(i)

geo_dict = (
    {
      labels[0]:f"Quaternary alluvium ({labels[0]})",
      labels[1]:f"Tertiary volcanic rocks, Columbia River Basalt Group ({labels[1]})",
      labels[2]:f"Tertiary volcanic rocks ({labels[2]})",
      labels[3]:f"Quaternary–Tertiary volcanic rocks ({labels[3]})",
      labels[4]:f"Pleistocene outburst-flood  deposits ({labels[4]})",
      labels[5]:f"Quaternary–Tertiary continental sedimentary rocks and deposits ({labels[5]})",
      labels[6]:f"Water ({labels[6]})",
      labels[7]:f"Holocene dune sand ({labels[7]})",
      labels[8]:f"Quaternary volcanic rocks ({labels[8]})",
      labels[9]:f"Quaternary mass-wasting deposits ({labels[9]})",
      labels[10]:f"Tertiary continental sedimentary rocks ({labels[10]})",
      labels[11]:f"Tertiary fragmental volcanic rocks ({labels[11]})",
      labels[12]:f"Tertiary intrusive rocks ({labels[12]})",
      labels[13]:f"Quaternary loess ({labels[13]})",
      labels[14]:f"Pleistocene alpine glacial drift ({labels[14]})",
      labels[15]:f"Quaternary–Tertiary intrusive rocks ({labels[15]})",
      labels[16]:f"Quaternary intrusive rocks ({labels[16]})",
      labels[17]:f"Quaternary fragmental volcanic rocks and deposits (includes lahars) ({labels[17]})",
      labels[18]:f"Mesozoic marine sedimentary rocks ({labels[18]})",
      labels[19]:f"Mesozoic intrusive rocks ({labels[19]})",
      labels[20]:f"Mesozoic metasedimentary and metavolcanic rocks ({labels[20]})",
      labels[21]:f"Mesozoic metasedimentary rocks ({labels[21]})",
      labels[22]:f"Mesozoic–Paleozoic metasedimentary and metavolcanic rocks ({labels[22]})",
      labels[23]:f"Tertiary nearshore sedimentary rocks ({labels[23]})",
      labels[24]:f"Ice ({labels[24]})",
      labels[25]:f"Tertiary marine sedimentary rocks ({labels[25]})",
      labels[26]:f"Tertiary volcanic rocks, Crescent Formation ({labels[26]})",
      labels[27]:f"Mesozoic orthogneiss ({labels[27]})",
      labels[28]:f"Mesozoic metavolcanic rocks ({labels[28]})",
      labels[29]:f"Mesozoic volcanic rocks ({labels[29]})",
      labels[30]:f"Pleistocene continental glacial drift ({labels[30]})",
      labels[31]:f"Tectonic zones; areas of intense cataclasis, including mylonitization ({labels[31]})",
      labels[32]:f"Mesozoic–Paleozoic heterogeneous metamorphic rocks ({labels[32]})",
      labels[33]:f"Precambrian metasedimentary rocks ({labels[33]})",
      labels[34]:f"Mesozoic–Paleozoic amphibolite ({labels[34]})",
      labels[35]:f"Precambrian heterogeneous metamorphic rocks ({labels[35]})",
      labels[36]:f"Paleozoic metavolcanic rocks ({labels[36]})",
      labels[37]:f"Mesozoic–Paleozoic ultramafic rocks ({labels[37]})",
      labels[38]:f"Mesozoic heterogeneous metamorphic rocks ({labels[38]})",
      labels[39]:f"Mesozoic gneiss ({labels[39]})",
      labels[40]:f"Tertiary–Cretaceous intrusive rocks ({labels[40]})",
      labels[41]:f"Paleozoic–Precambrian metasedimentary rocks ({labels[41]})",
      labels[42]:f"Mesozoic migmatite and mixed metamorphic and igneous rocks ({labels[42]})",
      labels[43]:f"Paleozoic metasedimentary rocks ({labels[43]})",
      labels[44]:f"Precambrian metavolcanic rocks ({labels[44]})",
      labels[45]:f"Precambrian intrusive rocks ({labels[45]})",
      labels[46]:f"Paleozoic intrusive rocks ({labels[46]})",
      labels[47]:f"Tertiary–Cretaceous orthogneiss ({labels[47]})",
      labels[48]:f"Paleozoic gneiss ({labels[48]})",
      labels[49]:f"Mesozoic continental sedimentary rocks ({labels[49]})",
      labels[50]:f"Mesozoic amphibolite ({labels[50]})",
      labels[51]:f"Paleozoic metasedimentary and metavolcanic rocks ({labels[51]})",
      labels[52]:f"Tertiary–Cretaceous gneiss ({labels[52]})",
      labels[53]:f"Mesozoic nearshore sedimentary rocks ({labels[53]})"
    }
)

In [29]:
"""Creating 3 common columns in all datasets
and assigning them the correct data"""

# I tried to do this pythonically and it didn't work

# def create_new_cols(gdf, gdf_dot_column, hazard_dict, haz_name, act_dict={}):
#  """creates 3 new columns on df so they have common cols
#  and assigns their relevant data"""
#   gdf["Risk"] = gdf_dot_column.map(hazard_dict)
#   if gdf != volcano:
#     gdf["Action"] = np.nan
#     gdf["Hazard"] = haz_name
#   if gdf == volcano:
#     gdf["Action"] = gdf_dot_column.map(act_dict)
#     gdf["Hazard"] = haz_name
#   return gdf

# tsunami = create_new_cols(tsunami, tsunami.TSUNAMI_SC, tsunami_dict, "Tsunami Risk")
# liquif = create_new_cols(liquif, liquif.LIQUEFAC_1, liquif_dict, "Liquifaction Susceptibility")
# volcano = create_new_cols(volcano, volcano.HAZARD_TYP, volcano_risk_dict, "Volcanic Risks", volcano_action_dict)
# geology = create_new_cols(geology, geology.Label, geo_dict, "Local Geology (no risk)")


# ------------------------------------
# I genuinely have no idea why this doesnt work in a function
# so here is it manually done

tsunami["Risk"] = tsunami.TSUNAMI_SC.map(tsunami_dict)
tsunami["Action"] = np.nan
tsunami["Hazard"] = "Tsunami Risk"

liquif["Risk"] = liquif.LIQUEFAC_1.map(liquif_dict)
liquif["Action"] = np.nan
liquif["Hazard"] = "Liquifaction Susceptibility"

volcano["Risk"] = volcano.HAZARD_TYP.map(volcano_risk_dict)
volcano["Action"] = volcano.HAZARD_TYP.map(volcano_actions_dict)
volcano["Hazard"] = "Volcanic Risks"

geology["Risk"] = geology.Label.map(geo_dict)
geology["Action"] = np.nan
geology["Hazard"] = "Local Geology (no risk)"

# GDF processing

In [30]:
"""Proces the hazard data:
- setting correct original coord reference system (crs)
- reset crs to allign with the crs of the address
- remove & rename all columns except the relevant ones"""

from pyproj import CRS

hazards_list = ["Tsunami Risk", "Liquifaction Susceptibility",
                "Volcanic Risks", "Local Geology (no risk)"]


def set_custom_crs(gdf):
  """sets custom crs
  Information from metadata"""
  custom_crs = CRS(
    proj="lcc", lat_1=47.33333333333334,
    lat_2=45.83333333333334, lat_0=45.33333333333334,
    lon_0=-120.5, x_0=500000,
    y_0=0, datum="NAD83 HARN",
    units="us-ft", ellps="GRS80")

  gdf = gdf.set_crs(custom_crs, allow_override=True)
  gdf = gdf.to_crs("EPSG:4326")
  return gdf


def relevant_columns(gdf):
  """keeps only the relevant columns
  of each gdf in the desired order"""
  gdf = gdf[["Hazard", "Risk", "Action", "geometry"]]
  return gdf


def gdfs_crs(gdfs):
  """sets correct crs for all gdfs"""
  crs_gdfs = []
  for gdf in gdfs:
    gdf2 = set_custom_crs(gdf)
    crs_gdfs.append(gdf2)
  return crs_gdfs


def filter_gdfs(gdfs):
  """keep only the relevant columns of all gdfs"""
  filtered_gdfs = []
  crs_gdfs = gdfs_crs(gdfs)
  for gdf in crs_gdfs:
    gdf2 = relevant_columns(gdf)
    filtered_gdfs.append(gdf2)
  return filtered_gdfs


def preprocess_gdfs():
  """pepares gdfs to needs"""
  gdfs_list = [tsunami, liquif, volcano, geology]
  gdfs = filter_gdfs(gdfs_list)
  return gdfs

In [31]:
"""find what risks are associates with the address"""

def find_overlap(address_gdf):
  """find which hazards intersect the address fromm all gdfs"""
  gdfs = preprocess_gdfs()
  overlap = []
  for gdf in gdfs:
    overlap.extend(gdf[
        gdf.intersects(address_gdf.unary_union)
        ].to_records(index=False))
  return overlap


def new_gdf(address_gdf):
  """creates a new gdf from data, renames columns"""
  overlap = find_overlap(address_gdf)
  haz_at_address = gpd.GeoDataFrame.from_records(overlap)
  haz_at_address =  haz_at_address.rename(
      columns={0:"Hazard", 1: "Risk", 2:"Action", 3:"geometry"})
  return haz_at_address

In [32]:
"""produce a printed report of the hazards found at the address"""

# associates simple names with official names. honestly idk, it made sense at the time
HAZARDS = {"tsunami":["Tsunami Risk", "tsunami inundation"],
           "liquifaction": ["Liquifaction Susceptibility", "liquifaction"],
           "volcano": ["Volcanic Risks", "volcanic activity"]}


def find_str_risk(gdf, hazard, x=0):
  """finds string of just the risk associated"""
  return gdf["Risk"][gdf["Hazard"] == hazard].drop_duplicates().values[x]


def print_actions(gdf, hazard, x):
  """prints the actions to perform for the risk
  volcanic risk is the only one with action items"""
  actions = gdf["Action"][gdf["Hazard"] == hazard].values[x]
  print(f'Recommended actions in the event of an eruption are as follows: \n')
  i = 1
  for action in actions:
    print(f'  {i}: {action} \n')
    i += 1


def risk_print_statements(counter, risk, gdf, hazard, x):
  """print statements for each risk"""
  print(f"Hazard {counter}: {risk.capitalize()}. \n")
  print(f'The {risk} risk factor at this address is: {find_str_risk(gdf, hazard, x)} \n')


def print_risks(gdf, hazard, risk, risk_count):
  """prints the risk statements for each hazard, and actions
  if applicable"""
  global NO_HAZ
  global COUNTER
  COUNTER = risk_count
  doubles = len(gdf["Risk"][gdf["Hazard"] == hazard].drop_duplicates())
  if any(gdf["Hazard"] == hazard):
    NO_HAZ = False
    for x in range(doubles):
      if hazard != "Volcanic Risks":
        risk_print_statements(COUNTER, risk, gdf, hazard, x)
        COUNTER += 1
      if hazard == "Volcanic Risks":
        risk_print_statements(COUNTER, risk, gdf, hazard, x)
        print_actions(gdf, hazard, x)
        COUNTER += 1
  else:
    print(f"There is no risk of {risk} at this address \n")
    NO_HAZ = True


def print_by_hazard(gdf, input_hazard, risk_count):
  """prints risk factor statements
  hazard options: tsunami, liquifaction or volcano"""
  try:
    hazard = HAZARDS.get(input_hazard)[0] # The hazard name as it appears in the df
    risk = HAZARDS.get(input_hazard)[1] # The risk as it will be written for the summary
    print_risks(gdf, hazard, risk, risk_count)
  except:
    print("There is an error")


def print_all_hazards(gdf):
  """iterates through hazards and prints all releavnt to address"""
  hazards = ["tsunami", "liquifaction", "volcano"]
  print(f'There are {len(gdf["Risk"].drop_duplicates())-1} hazards & risks found at {ADDRESS}: \n') # -1 because geology is not a risk
  risk_count = 1
  for hazard in hazards:
    print_by_hazard(gdf, hazard, risk_count)
    if NO_HAZ == False:
      risk_count = COUNTER


def print_geology(gdf):
  """prints geology at address"""
  geo = gdf["Risk"][gdf["Hazard"] == "Local Geology (no risk)"].values[0]
  print(f'The geology of {ADDRESS} is: {geo}. \n*Note: This is not a hazard or risk, but may be useful to know \n')


def print_hazards_main(address_gdf):
  """main execution of processing the gdfs and printing
  the hazards associated with the address"""
  gdf = new_gdf(address_gdf)
  print_geology(gdf)
  print_all_hazards(gdf)

In [33]:
def main():
  """takes a users address and returns
  what hazards are documented at the address"""
  address_gdf = confirmed_address()
  print("")
  print_hazards_main(address_gdf)

In [ ]:
main()

Sample addresses:
-	1 of each: 3 E Park St, Bay Center, WA 98527, United States
-	Multi-Volcanic, no tsunami: 24000 Spirit Lake Hwy, Toutle, WA 98649, United States
-	Multi-Tsunami: 48.33348176083236, -124.66031049466368
-	Invalid geocode: 13427 446th ave se north bend, WA, 98045, United States
-	Valid geocode: 13427 446th ave se
-	Feel free to try out any other Washington State addresses or coordinates. In some cases, such as the two geocode examples above, information beyond the street address can result in invalid geocoding. Removing city, zip, country usually seems to work, but coordinates always work too.

